In [1]:
import numpy as np
import pandas as pd
import os
import geopandas as gpd
from datetime import date
from pathlib import Path
import pickle
import codebase
from codebase import ml_pipeline

- US-Mexico border: 4152050
- below Hoover Dam: 4152103
- Lee's Ferry: 4152450
- upstream of Lake Powell (San Juan River trib): 4152600

In [2]:
## define experimental set-up

# grdc stored as floats in the downlaod jsons
grdc_id = 1834101
grdc_sub_ids = []#[4152450,4152600] ## MUST BE ORDERED DOWNSTREAM (first) TO UPSTREAM (last)
dam_name = 'kainji'
start_year = 2010
stop_year_ex = 2024
basin_str = 'niger'

In [3]:
## Other variables and filepaths
grdc_dir = "/global/scratch/users/ann_scheliga/aux_dam_datasets/GRDC_CRB/"
met_dir = "/global/scratch/users/ann_scheliga/era5_data/"
res_dir = "/global/scratch/users/ann_scheliga/CYGNSS_daily/time_series/"
basin_data_dir = "/global/scratch/users/ann_scheliga/basin_forcing_processed/"

In [ ]:
test_df = ml_pipeline.LSTM_preprocessing_nh(grdc_id,
    grdc_sub_ids,
    dam_name = 'glen canyon',
    start_year = 2018,
    stop_year_ex = 2024,
    save_output=False,
    basin_str=basin_str,
    res_dir=res_dir
    )

## testing EDA plotting

In [ ]:
precip_cols = test_df.columns[test_df.columns.str.contains("precip")]
tempK_cols = test_df.columns[test_df.columns.str.contains("tempK")]
dewpoint_cols = test_df.columns[test_df.columns.str.contains("dew")]

In [ ]:
test_df[dewpoint_cols].plot()

In [ ]:
test_df[tempK_cols].plot()

## pre-processing step-by-step

In [4]:
# For debugging
def check_data_format(df):
    print(df.head(2))
    print(df.tail(2))
    print('structure type:',type(df))
    print('index type:',type(df.index))
    print('first index:',df.index[0])
    print('Inferred frequency:',df.index.inferred_freq)


In [ ]:
# Create output dataframe
full_time = pd.date_range(start=date(start_year,1,1), end=date(stop_year_ex,1,1),freq='D')
output_df = pd.DataFrame(index = full_time)

In [ ]:
## import sw_area
sw_area = codebase.load_data.load_daily_reservoir_CYGNSS_area(
    dam_name, filepath=res_dir
)
output_df['SW_area'] = sw_area
output_df['SW_area'].fillna(output_df['SW_area'].mean(),inplace=True)

check_data_format(output_df['SW_area'])

In [ ]:
## Calculate SW_flag
output_df['SW_flag'] = 0
# where SW_area has a value, SW_flag is true
output_df.loc[~output_df['SW_area'].isna(),'SW_flag'] = 1 

check_data_format(output_df['SW_flag'])

In [ ]:
grdc_Q.shape

In [ ]:
## import GRDC
watershed_gpd, grdc_Q = codebase.load_data.load_GRDC_station_data_by_ID(
    grdc_id,
    filepath=grdc_dir,
    timeseries_dict={"start_year": start_year, "stop_year": stop_year_ex},
    basin_str='niger'
)

output_df['Q'] = grdc_Q
check_data_format(grdc_Q)

In [ ]:
# Given that sub-basins exist
subbasins_GRDC = list(map(
    lambda id: codebase.load_data.load_GRDC_station_data_by_ID(
        id,filepath=grdc_dir,
        timeseries_dict={"start_year": start_year, "stop_year": stop_year_ex}
        ),
    grdc_sub_ids))
# subbasin_zipped = dict(zip(grdc_sub_ids,subbasins_GRDC))

# drop flow timeseries tuple from list, leave just the geoDataFrame(s)
subbasin_shps = [output[0] for output in subbasins_GRDC]
def create_XOR_subasins(list_of_shps,base_gpd):
    processed_shps = base_gpd.iloc[:,-1]
    processed_shps.index = ['_ex0']
    processed_shps.index.rename('relative_order',inplace=True)

    for idx, shp in enumerate(list_of_shps):
        shp_to_diff = shp.iloc[0,-1]
        processed_shps = processed_shps.difference(shp_to_diff)
        processed_shps.loc['_ex'+str(idx+1)] = shp_to_diff
    
    # Using the .difference() method wth a shapely shape removes the crs
    processed_shps.set_crs(base_gpd.crs,inplace=True)

    return processed_shps

In [ ]:
XOR_geoms = create_XOR_subasins(subbasin_shps,watershed_gpd)

In [ ]:
# Sanity check, not essentia
print(XOR_geoms.index)
XOR_geoms.boundary.plot(cmap='viridis')
XOR_geoms[0]

In [ ]:
## Can't get mode function to work easily.
# Tried scipy and statistics modules
# type_precip_test = codebase.area_subsets.era5_shape_subset_and_concat_from_file_pattern(
#     filepath = met_dir,
#     input_pattern = r'daily_precip_type',
#     subset_gpd = watershed_gpd,
#     concat_dict = concat_dict,
#     agg_function = mode
# )

In [ ]:
subsets_gpd = pd.concat([watershed_gpd,*subbasin_shps])
subsets_geoms = subsets_gpd['geometry'].reset_index(drop=True).add_prefix('_tot')
subsets_geoms[0]

In [ ]:
all_shps = pd.concat([subsets_geoms,XOR_geoms]).rename('geometry')#.reset_index()
all_shps

In [ ]:
def add_era5_met_data_by_shp(input_gpd,met_dir,col_suffix = "",start_year=-1,stop_year_ex=-1):
    """
    Load areal aggregated temp and precip based on provided gpd.

    Long Description
    ----------------
    Uses `area_subsets.era5_shape_subset_and_concat_from_file_pattern` for each variable.
    Searches for all instances of a substring (ex: 'daily_tempK'),
    and concatenates all found files according to hard-coded concat_dict dimensions.
    For precipitation, aggregates using np.nansum
    For temperature, aggregates using np.nanmean.
    start and stop year not used, but included in case useful in future edits.

    Inputs
    ------
    input_gpd : geopandas.GeoDataFrame
        geometry to subset data
    col_suffix : str
        default = "" (empty)
        added to column names in final output dataframe
        useful when using this function multiple times
    start_year, stop_year_ex : int
        default = -1
        not used, passed in case future edits need the bounds
        for pattern parsing or filtering.
    """
    
    concat_dict = {"dim": "valid_time"}

    __ , tempK_1dim = codebase.area_subsets.era5_shape_subset_and_concat_from_file_pattern(
        filepath = met_dir,
        input_pattern = r'daily_tempK',
        subset_gpd = input_gpd,
        concat_dict = concat_dict,
        agg_function = np.nanmean
    )
    tempK_1dim.rename('tempK',inplace=True)

    __ , precip_1dim = codebase.area_subsets.era5_shape_subset_and_concat_from_file_pattern(
        filepath = met_dir,
        input_pattern = r'daily_tot_precip',
        subset_gpd = input_gpd,
        concat_dict = concat_dict,
        agg_function = np.nansum
    )
    precip_1dim.rename('precipm',inplace=True)
    met_df = pd.concat([tempK_1dim, precip_1dim],axis=1).add_suffix(col_suffix)
    return met_df

In [ ]:
met_list = list(map(lambda idx: add_era5_met_data_by_shp(all_shps.loc[[idx]],met_dir=met_dir,col_suffix=idx),all_shps.index[:2]))

In [ ]:
met_df = pd.concat(met_list,axis=1)
output_df = output_df.join(met_df, how='left')
output_df

In [ ]:
output_df.interpolate(
        method="linear", axis=0, inplace=True  , limit=7
    )  # interpolate missing interior values
# output_df.bfill(inplace=True, limit=2)  # backfill missing first precip value
output_df.index.name = 'date'

In [ ]:
output_df

In [ ]:
output_dict= {grdc_id:output_df}
filename = str(grdc_id)+'.pkl'
pickle.dump(output_dict, open(basin_data_dir+filename, 'wb'))

## Manual data processing
Replaced the SW_area nan values with the mean. Did not rerun the full pre-processing script. Manually overwrote. Pre-processing script contains the updated .fillna() fix.

In [ ]:
# all_pkls = [fn for fn in os.listdir(basin_data_dir) if '.pkl' in fn]
# all_pkls.sort()
def replace_SW_nan_with_mean(fn):
    test_read = pickle.load(open(basin_data_dir+fn, 'rb'))
    grdc_id = list(test_read.keys())[0]
    print('GRDC ID:',grdc_id)
    output_df = test_read[grdc_id]
    sw_mean = output_df["SW_area"].mean()
    print('SW_mean:',sw_mean)
    output_df["SW_area"].fillna(sw_mean, inplace=True)
    check_data_format(output_df)

    output_dict= {grdc_id:output_df}
    pickle.dump(output_dict, open(basin_data_dir+fn, 'wb'))
# replace_SW_nan_with_mean(all_pkls[8])

GRDC ID: 5608096
SW_mean: 1109.5844779328816
                SW_area  SW_flag       Q  max_tempK_tot0  min_tempK_tot0  \
date                                                                       
2000-01-01  1109.584478        0  53.225      300.054321      294.764191   
2000-01-02  1109.584478        0  52.407      298.432037      294.155884   

            precipm_tot0  dewpointK_tot0  max_tempK_tot1  min_tempK_tot1  \
date                                                                       
2000-01-01      0.128849      293.081573      300.573181      294.206329   
2000-01-02      0.450547      292.195190      297.472382      293.030182   

            precipm_tot1  dewpointK_tot1  max_tempK_ex0  min_tempK_ex0  \
date                                                                     
2000-01-01      0.031355      292.013092     299.755280     294.996216   
2000-01-02      0.172825      291.009125     299.237671     294.684052   

            precipm_ex0  dewpointK_ex0  max_temp